In [38]:
#!pip install requests
#!pip install pandas
#!pip install linkml_runtime


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 6.8 MB/s eta 0:00:00m eta 0:00:010:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached hbreader-0.9.1-py3-none-any.whl (7.6 kB)
  Using cached json_flattener-0.1.9-py3-none-any.whl (10 kB)
  Using cached jsonasobj2-1.0.4-py3-none-any.whl (6.3 kB)
  Using cached prefixcommons-0.1.12-py3-none-any.whl (29 kB)
  Using cached prefixmaps-0.1.5-py3-none-any.whl (232 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 24.0 MB/s eta 0:00:00
  Using cached rdflib-7.0.0-py3-none-any.whl (531 kB)
  Using cached PyTrie-0.4.0-py3-none-any.whl
  Using cached pytest_logging-2015.11.4-py3-none-any.whl
  Using cached greenlet-2.0.1.

In [1]:
import requests
import pandas as pd
from linkml_runtime import SchemaView
from linkml_runtime.utils.formatutils import camelcase

biolink = SchemaView(
    f"https://raw.githubusercontent.com/biolink/biolink-model/v3.5.4/biolink-model.yaml")

v2_edge_url = 'http://solr.monarchinitiative.org/solr/golr/select?wt=json&q=*:*&rows=0&facet=true'
v2_node_url = 'http://solr.monarchinitiative.org/solr/search/select?wt=json&q=*:*&rows=0&facet=true'

In [2]:
node_pivot = "category,taxon_label"
node_response = requests.get(f"{v2_node_url}&facet.pivot={node_pivot}").json()['facet_counts']['facet_pivot'][node_pivot]


node_counts = []
for category in node_response:     
    if 'pivot' in category:
        for taxon in category['pivot']:        
            node_counts.append({
                "category": category["value"],
                "taxon_label": taxon['value'],
                "count": taxon['count']
            })
    else:
        node_counts.append({
            "category": category["value"],
            "taxon_label": None,
            "count": category['count']
        })

node_counts_df = pd.DataFrame(node_counts)
node_counts_df

,category,taxon_label,count
0,sequence feature,Homo sapiens,664846
1,sequence feature,Mus musculus,306907
2,sequence feature,Drosophila melanogaster,224482
3,sequence feature,Bos taurus,204846
4,sequence feature,Caenorhabditis elegans,76760
...,...,...,...
394,case,None,39
395,inheritance,None,32
396,association,None,2
397,dataset,None,1


In [3]:

def convert_node_category(value: str):
    if biolink.get_class(value):
        return 'biolink:' + camelcase(biolink.get_class(value).name)

    for k,v in biolink.all_aliases().items():
        if value in v:
            return 'biolink:' + camelcase(k)

    mappings = {
        'variant': 'biolink:SequenceVariant',
        'sequence alteration': 'biolink:SequenceVariant',
        'quality': 'biolink:PhenotypicFeature',
        'cellular process': 'biolink:BiologicalProcess'
    }
    if value in mappings:
        return mappings[value]
    
    return None

node_counts_df["biolink_category"] = node_counts_df['category'].apply(convert_node_category)

node_counts_df[node_counts_df['biolink_category'].isna()]["category"].unique()

array(['variant locus', 'phenotypic_marker', 'evidence', 'chromosome',
       'ObjectProperty', 'clinical_course', 'AnnotationProperty',
       'DatatypeProperty'], dtype=object)

In [4]:
node_counts_df

,category,taxon_label,count,biolink_category
0,sequence feature,Homo sapiens,664846,biolink:NucleicAcidEntity
1,sequence feature,Mus musculus,306907,biolink:NucleicAcidEntity
2,sequence feature,Drosophila melanogaster,224482,biolink:NucleicAcidEntity
3,sequence feature,Bos taurus,204846,biolink:NucleicAcidEntity
4,sequence feature,Caenorhabditis elegans,76760,biolink:NucleicAcidEntity
...,...,...,...,...
394,case,None,39,biolink:Case
395,inheritance,None,32,biolink:GeneticInheritance
396,association,None,2,biolink:Association
397,dataset,None,1,biolink:Dataset


In [5]:
response = requests.get(v2_edge_url + "&facet.pivot=association_type").json()['facet_counts']['facet_pivot']['association_type']
edge_counts_df = pd.DataFrame(response).drop("field",axis=1).rename(columns={"value":"category"})

def convert_edge_category(value: str):
    mappings = {
        'gene_homology': 'biolink:GeneToGeneHomologyAssociation',
        'publication_gene': 'biolink:InformationContentEntityToNamedThingAssociation',
        'publication_variant': 'biolink:InformationContentEntityToNamedThingAssociation',
        'gene_interaction': 'biolink:PairwiseGeneToGeneInteraction',
        'gene_function': 'biolink:_Association',
        'gene_anatomy': 'biolink:_Association',
        'variant_phenotype': 'biolink:_Association',
        'variant_gene': 'biolink:_Association',
        'gene_phenotype': 'biolink:_Association',
        'model_gene': 'biolink:_Association',
        'pathway_phenotype': 'biolink:_Association',
        'variant_genotype': 'biolink:_Association',
        'genotype_phenotype': 'biolink:_Association',
        'publication_phenotype': 'biolink:_Association',
        'gene_pathway': 'biolink:_Association',
        'publication_disease': 'biolink:_Association',
        'genotype_gene': 'biolink:_Association',
        'variant_disease': 'biolink:_Association',
        'disease_phenotype': 'biolink:_Association',
        'publication_genotype': 'biolink:_Association',
        'publication_model': 'biolink:_Association',
        'model_variant': 'biolink:_Association',
        'model_disease': 'biolink:_Association',
        'disease_pathway': 'biolink:_Association',
        'model_case': 'biolink:_Association',
        'marker_disease': 'biolink:_Association',
        'chemical_disease': 'biolink:_Association',
        'genotype_disease': 'biolink:_Association',
        'gene_disease': 'biolink:_Association',
        'case_genotype': 'biolink:_Association',
        'model_genotype': 'biolink:_Association',
        'case_phenotype': 'biolink:_Association',
        'case_variant': 'biolink:_Association',
        'case_gene': 'biolink:_Association',
    }
    if value in mappings:
        return mappings[value]

    return None

edge_counts_df["biolink_category"] = edge_counts_df['category'].apply(convert_edge_category)
for category in edge_counts_df["category"]:
    print(f"'{category}': 'biolink:_Association',")

# edge_counts_df

'gene_homology': 'biolink:_Association',
'publication_gene': 'biolink:_Association',
'publication_variant': 'biolink:_Association',
'gene_interaction': 'biolink:_Association',
'gene_function': 'biolink:_Association',
'gene_anatomy': 'biolink:_Association',
'variant_phenotype': 'biolink:_Association',
'variant_gene': 'biolink:_Association',
'gene_phenotype': 'biolink:_Association',
'model_gene': 'biolink:_Association',
'pathway_phenotype': 'biolink:_Association',
'variant_genotype': 'biolink:_Association',
'genotype_phenotype': 'biolink:_Association',
'publication_phenotype': 'biolink:_Association',
'gene_pathway': 'biolink:_Association',
'publication_disease': 'biolink:_Association',
'genotype_gene': 'biolink:_Association',
'variant_disease': 'biolink:_Association',
'disease_phenotype': 'biolink:_Association',
'publication_genotype': 'biolink:_Association',
'publication_model': 'biolink:_Association',
'model_variant': 'biolink:_Association',
'model_disease': 'biolink:_Association',
'di

In [6]:
["biolink:" + camelcase(c) for c in biolink.all_classes() if "gene to gene" in c.lower()]

['biolink:GeneToGeneAssociation',
 'biolink:GeneToGeneHomologyAssociation',
 'biolink:GeneToGeneFamilyAssociation',
 'biolink:GeneToGeneCoexpressionAssociation',
 'biolink:PairwiseGeneToGeneInteraction',
 'biolink:GeneToGeneProductRelationship']